In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from validate_email import validate_email
from bs4.element import Comment
# import urllib.request


In [2]:
USER_AGENT = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
def fetch_results(search_term, number_results, language_code,sup_page=None):
    assert isinstance(search_term, str), 'Search term must be a string'
    assert isinstance(number_results, int), 'Number of results must be an integer'

    escaped_search_term = search_term.replace(' ', '+')
    if type(sup_page)==str:
        sup_page_term = sup_page.replace(' ', '+')
        escaped_search_term = escaped_search_term + '+' + sup_page_term

    google_url = 'https://www.google.com/search?q={}&num={}&hl={}'.format(escaped_search_term, number_results, language_code)
    response = requests.get(google_url, headers=USER_AGENT)
    response.raise_for_status()
 
    return search_term, response.text
def parse_results(html, keyword):
    soup = BeautifulSoup(html, 'html.parser')
 
    found_results = pd.DataFrame(columns = ['keyword', 'title', 'rank', 'link'])
    rank = 1
    result_block = soup.find_all('div', attrs={'class': 'g'})
    for result in result_block:
 
        link = result.find('a', href=True)
        title = result.find('h3')
        #description = result.find('span', attrs={'class': 'st'})
        if link and title:
            link = link['href']
            title = title.get_text()
            #if description:
            #    description = description.get_text()
            if link != '#':
                found_results=found_results.append({'keyword': keyword,'title':title,'rank':rank,'link':link},ignore_index=True)
                rank += 1
    return found_results
def scrape_google(search_term, number_results, language_code,sup_page=None,info=["Điện thoại", "Email"]):
    try:
        keyword, html = fetch_results(search_term, number_results, language_code,sup_page)
        results_list = parse_results(html, keyword)
        
        return results_list
    except AssertionError:
        raise Exception("Incorrect arguments parsed to function")
    except requests.HTTPError:
        raise Exception("You appear to have been blocked by Google")
    except requests.RequestException:
        raise Exception("Appears to be an issue with your connection")
def get_info_wiki(link,info=["Điện thoại", "Email"]):
    request = requests.get(link)
    html = BeautifulSoup(request.text)
    table = html.find_all('table',{'class':'infobox'})[0]
    tr_list = table.find_all('tr')
    result = {}
    for tr in tr_list:
        if tr.find('th'):
            if tr.find('th').text in info:
                result[tr.find('th').text] = tr.find('td').text
    return result

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    #soup = BeautifulSoup(body, 'html.parser')
    texts = body.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def getFullText(html,part='body'):
    texts =html.find(part)
    texts = text_from_html(texts)
    return texts
    
def getVaildEmail(texts):
    texts_list = texts.split(" ")
    mail_list = []
    for text in texts_list:
        if validate_email(text):
            mail_list.append(text)
    return mail_list
def getPhoneNumber(texts):
    texts = texts.split(" ")
    number_phones = []
    for i in range(len(texts)):
        text = texts[i].replace(".","")
        if 7<=len(text)<=11 and text.isdigit():
            number_phones.append(text)
            continue
        if "84" in texts[i]:
            if texts[i+1].isdigit():
                if text[i+2].isdigit():
                    number_phones.append(texts[i] + texts[i+1] + texts[i+2])
                else:
                    number_phones.append(texts[i] + texts[i+1])
    return number_phones

In [3]:
results = scrape_google('"đại học" email và số số điện thoại & "liên hệ"',10,'vi')
results

,keyword,title,rank,link
0,"""đại học"" email và số số điện thoại & ""liên hệ""","Hướng dẫn kiểm tra thông tin, sửa email và SĐT...",1,https://tuyensinhso.vn/tuyen-sinh-247/huong-da...
1,"""đại học"" email và số số điện thoại & ""liên hệ""","Danh sách số điện thoại, email Ban chấp hành Đ...",2,http://nute.edu.vn/ChitietTin.aspx?id=2498
2,"""đại học"" email và số số điện thoại & ""liên hệ""",Liên hệ - Đại học Quốc tế,3,http://www2.hcmiu.edu.vn/Lien-He-Contact-Us
3,"""đại học"" email và số số điện thoại & ""liên hệ""",Số điện thoại và địa chỉ Email hỗ trợ giải đáp...,4,https://tdmu.edu.vn/thong-bao/tuyen-sinh/so-di...
4,"""đại học"" email và số số điện thoại & ""liên hệ""",Danh bạ điện thoại nội bộ - Trường Đại học Kho...,5,https://www.hcmus.edu.vn/phone
5,"""đại học"" email và số số điện thoại & ""liên hệ""",Tôi đã bị mất số điện thoại. và giờ tôi bị mất...,6,https://support.google.com/mail/forum/AAAAz2mQ...
6,"""đại học"" email và số số điện thoại & ""liên hệ""",Thông tin hỗ trợ công tác thi và tuyển sinh đạ...,7,https://vnu.edu.vn/ttsk/?C2097/N21991/Thong-ti...
7,"""đại học"" email và số số điện thoại & ""liên hệ""",Liên hệ - University of Economics Ho Chi Minh ...,8,https://www.ueh.edu.vn/lien-he
8,"""đại học"" email và số số điện thoại & ""liên hệ""",Danh bạ điện thoại - trung tâm tin học - đại h...,9,https://cict.agu.edu.vn/danhbadienthoai.cict
9,"""đại học"" email và số số điện thoại & ""liên hệ""",Đại học quốc gia TP Hồ Chí Minh - Đại học Quốc...,10,https://vnuhcm.edu.vn/lien-he


In [4]:
info_list = pd.DataFrame(columns=['title','email',"sdt"])
for link,title in zip(results['link'],results['title']):
    try:
        request = requests.get(link)
        html = BeautifulSoup(request.text, "lxml")
        texts = getFullText(html)
        try:
            emails = getVaildEmail(texts)
            emails = list(set(emails))
        except:
            pass
        try:
            number_phone = getPhoneNumber(texts)
            number_phone = list(set(number_phone))
        except:
            pass
        info_list=info_list.append({'title':title,'email':emails,'sdt':number_phone},ignore_index=True)
    except:
        pass
    

In [5]:
info_list['email'] = info_list['email'].str.join(',')
info_list['sdt'] = info_list['sdt'].str.join(',')
info_list

,title,email,sdt
0,"Hướng dẫn kiểm tra thông tin, sửa email và SĐT...",tuyensinhso.com@gmail.com,0889964368
1,"Danh sách số điện thoại, email Ban chấp hành Đ...","onlylove165@gmail.com,tranthuy.nute@gmail.com,...","01634229954,0917749254,0904770053,0973776072,0..."
2,Liên hệ - Đại học Quốc tế,info@hcmiu.edu.vn,"37244271,37244270"
3,Số điện thoại và địa chỉ Email hỗ trợ giải đáp...,tuyensinh@tdmu.edu.vn,"3835677,3844340,3844341"
4,Danh bạ điện thoại nội bộ - Trường Đại học Kho...,,"73089899,62884499"
5,Tôi đã bị mất số điện thoại. và giờ tôi bị mất...,,01652727319
6,Liên hệ - University of Economics Ho Chi Minh ...,"cdea@ueh.edu.vn,inkinhte@ueh.edu.vn,yte@ueh.ed...","38561249,35265829,38231606,38295635,38292170,3..."
7,Danh bạ điện thoại - trung tâm tin học - đại h...,cict@agu.edu.vn,"38561249,35265829,38231606,38295635,38292170,3..."
